# Model-Observation comparison: comparing to the World Ocean Database

The goal of this tutorial is to get familiar with the basics of using CrocoCamp to interpolate MOM6 output onto the space of observations stored in the World Ocean Database (WOD). For this tutorial we will use output from a MOM6 run already stored on NCAR's HPC system, and WOD13 observation files already in obs_sequence format and also stored on NCAR's HPC system.

## The configuration file

To interpolate the model data onto the observation space, CrocoCamp needs to know where the model and observation data are. We can tell it by using a `config.yaml` configuration file.

Start by copying the template into a new file (e.g. in the command line and from this notebook's directory: `cp ../configs/config_template.yaml ../configs/config_my_tutorial_3.yaml`)

We then use our favourite editor to modify the new file, making sure that the paths in the `INPUT FILES` section are as follows (note that the MOM6 files are the same as in tutorial 1, so those paths are identical between the two tutorials):

Note that `template_file` must be any of the output model files, `static_file` and `ocean_geometry` are the respective MOM6 files containing static and ocean geometry information.

Now give a look at the paths in the `OUTPUT FILES` section, and make sure that you are ok with the defaults or change them. You can ignore the `DART` section for this tutorial.

## Running the workflow

Running the workflow to interpolate the model is quite simple: create a `WorkflowModelObs` instance using the config file that you just generated, and then call its `run()` method. If you want to double check that your config file is correct, you can look at the solution `../configs/config_tutorial_3.yaml`.

While running, CrocoCamp generates temporary input files that tell DART's `perfect_model_obs` executable where to find MOM6 and CrocoLake data to perform the interpolation.

In [ ]:
from crococamp.workflows import WorkflowModelObs

# Create and run workflow to interpolate MOM6 model onto World Ocean Database obs space
workflow_WOD13 = WorkflowModelObs.from_config_file('../configs/config_my_tutorial_3.yaml')
workflow_WOD13.run(clear_output=True) #use flag clear_output=True if you want to re-run it and automatically clean all previous output

## Displaying the interactive map

CrocoCamp generates a parquet dataset that contains the values of the WOD observations, the MOM6 model data interpolated onto the observations space, and some basic statistics.

You can load and explore the data using pandas (CrocoCamp supports also dask for large datasets).

In [ ]:
import pandas as pd
# Load the parquet dataset generated by the workflow above
# This contains model data interpolated onto observation locations plus diagnostics
df = pd.read_parquet(workflow_WOD13.config['parquet_folder']) # config contains the information from the config file
df.head() # displays first 5 rows in the dataframe

Loading the interactive map is as simple as importing the widget and passing the dataframe to it:

In [ ]:
from crococamp.viz import InteractiveWidgetMap
# Create an interactive map widget to visualize model-observation comparisons
# The widget provides controls for selecting variables, observation types, and time ranges
widget = InteractiveWidgetMap(df)
widget.setup()